In [1]:
from langchain_groq import ChatGroq
from langchain.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_community.tools import DuckDuckGoSearchRun
import requests
import json
from typing import Dict, List, Optional
from dataclasses import dataclass
from datetime import datetime, timedelta

#### Data models for travel information

In [2]:
@dataclass
class TravelRequest:
    """User's travel request details"""
    destination: str
    duration_days: int
    budget_range: str  # e.g., "budget", "mid-range", "luxury"
    travelers: int
    user_currency: str
    departure_date: str

@dataclass
class WeatherInfo:
    """Weather information structure"""
    current_temp: float
    forecast: List[Dict]
    description: str

@dataclass
class CostBreakdown:
    """Detailed cost breakdown"""
    accommodation: float
    food: float
    activities: float
    transportation: float
    total_usd: float
    total_user_currency: float
    daily_budget: float

@dataclass
class TravelPlan:
    """Complete travel plan structure"""
    destination: str
    duration: int
    weather: WeatherInfo
    attractions: List[str]
    restaurants: List[str]
    activities: List[str]
    accommodation_options: List[str]
    cost_breakdown: CostBreakdown
    itinerary: Dict[str, List[str]]
    summary: str

#### Classes to define the Tools

In [ ]:
class WeatherTools:
    """Weather-related tools for travel planning"""
    
    @staticmethod
    @tool
    def get_current_weather(city: str) -> str:
        """
        Get current weather information for a city.
        
        Args:
            city (str): Name of the city
            
        Returns:
            str: Current weather information
        """
        try:
            search = DuckDuckGoSearchRun()
            query = f"current weather {city} temperature"
            weather_info = search.run(query)
            return f"Current weather in {city}: {weather_info[:200]}"
        except Exception as e:
            return f"Unable to fetch weather for {city}: {str(e)}"
    
    @staticmethod
    @tool
    def get_weather_forecast(city: str, days: int = 7) -> str:
        """
        Get weather forecast for a city.
        
        Args:
            city (str): Name of the city
            days (int): Number of days for forecast
            
        Returns:
            str: Weather forecast information
        """
        try:
            search = DuckDuckGoSearchRun()
            query = f"weather forecast {city} next {days} days"
            forecast_info = search.run(query)
            return f"Weather forecast for {city} ({days} days): {forecast_info[:300]}"
        except Exception as e:
            return f"Unable to fetch forecast for {city}: {str(e)}"

In [4]:
class AttractionTools:
    """Tools for finding attractions, restaurants, and activities"""
    
    @staticmethod
    @tool
    def search_attractions(city: str) -> str:
        """
        Search for top attractions in a city.
        
        Args:
            city (str): Name of the city
            
        Returns:
            str: List of top attractions
        """
        try:
            search = DuckDuckGoSearchRun()
            query = f"top tourist attractions {city} must visit places"
            attractions = search.run(query)
            return f"Top attractions in {city}: {attractions[:400]}"
        except Exception as e:
            return f"Unable to find attractions for {city}: {str(e)}"
    
    @staticmethod
    @tool
    def search_restaurants(city: str, budget: str = "mid-range") -> str:
        """
        Search for restaurants in a city based on budget.
        
        Args:
            city (str): Name of the city
            budget (str): Budget level (budget, mid-range, luxury)
            
        Returns:
            str: List of recommended restaurants
        """
        try:
            search = DuckDuckGoSearchRun()
            query = f"best {budget} restaurants {city} local food"
            restaurants = search.run(query)
            return f"Recommended {budget} restaurants in {city}: {restaurants[:400]}"
        except Exception as e:
            return f"Unable to find restaurants for {city}: {str(e)}"
    
    @staticmethod
    @tool
    def search_activities(city: str) -> str:
        """
        Search for activities and experiences in a city.
        
        Args:
            city (str): Name of the city
            
        Returns:
            str: List of activities and experiences
        """
        try:
            search = DuckDuckGoSearchRun()
            query = f"things to do activities experiences {city}"
            activities = search.run(query)
            return f"Activities in {city}: {activities[:400]}"
        except Exception as e:
            return f"Unable to find activities for {city}: {str(e)}"

In [5]:
class AccommodationTools:
    """Tools for hotel and accommodation search"""
    
    @staticmethod
    @tool
    def search_hotels(city: str, budget_range: str) -> str:
        """
        Search for hotels in a city based on budget range.
        
        Args:
            city (str): Name of the city
            budget_range (str): Budget category
            
        Returns:
            str: Hotel options with approximate prices
        """
        try:
            search = DuckDuckGoSearchRun()
            query = f"hotels {city} {budget_range} price per night"
            hotels = search.run(query)
            return f"Hotel options in {city} ({budget_range}): {hotels[:400]}"
        except Exception as e:
            return f"Unable to find hotels for {city}: {str(e)}"
    
    @staticmethod
    @tool
    def estimate_accommodation_cost(city: str, days: int, budget_range: str) -> str:
        """
        Estimate total accommodation cost for the trip.
        
        Args:
            city (str): Destination city
            days (int): Number of days
            budget_range (str): Budget category
            
        Returns:
            str: Estimated accommodation cost
        """
        # Rough estimates based on budget range
        daily_rates = {
            "budget": 50,
            "mid-range": 120,
            "luxury": 300
        }
        
        base_rate = daily_rates.get(budget_range.lower(), 120)
        total_cost = base_rate * days
        
        return f"Estimated accommodation cost for {days} days in {city} ({budget_range}): ${total_cost} (${base_rate}/night)"

In [6]:
class CalculationTools:
    """Mathematical tools for cost calculations"""
    
    @staticmethod
    @tool
    def add_costs(*costs: float) -> float:
        """
        Add multiple costs together.
        
        Args:
            *costs: Variable number of cost values
            
        Returns:
            float: Sum of all costs
        """
        return sum(costs)
    
    @staticmethod
    @tool
    def multiply_cost(base_cost: float, multiplier: float) -> float:
        """
        Multiply a cost by a factor.
        
        Args:
            base_cost (float): Base cost amount
            multiplier (float): Multiplication factor
            
        Returns:
            float: Calculated result
        """
        return base_cost * multiplier
    
    @staticmethod
    @tool
    def calculate_daily_budget(total_cost: float, days: int) -> float:
        """
        Calculate daily budget from total cost.
        
        Args:
            total_cost (float): Total trip cost
            days (int): Number of days
            
        Returns:
            float: Daily budget amount
        """
        return total_cost / days if days > 0 else 0

In [7]:
class CurrencyTools:
    """Currency conversion tools"""
    
    @staticmethod
    @tool
    def get_exchange_rate(from_currency: str, to_currency: str) -> str:
        """
        Get current exchange rate between two currencies.
        
        Args:
            from_currency (str): Source currency code (e.g., USD)
            to_currency (str): Target currency code (e.g., EUR)
            
        Returns:
            str: Exchange rate information
        """
        try:
            search = DuckDuckGoSearchRun()
            query = f"exchange rate {from_currency} to {to_currency} today"
            rate_info = search.run(query)
            return f"Exchange rate {from_currency} to {to_currency}: {rate_info[:200]}"
        except Exception as e:
            return f"Unable to get exchange rate: {str(e)}"
    
    @staticmethod
    @tool
    def convert_currency(amount: float, from_currency: str, to_currency: str, rate: float = None) -> str:
        """
        Convert amount from one currency to another.
        
        Args:
            amount (float): Amount to convert
            from_currency (str): Source currency
            to_currency (str): Target currency
            rate (float): Exchange rate (if known)
            
        Returns:
            str: Converted amount
        """
        if rate is None:
            # Simplified conversion - in real app, fetch live rates
            common_rates = {
                "USD_EUR": 0.92,
                "USD_GBP": 0.79,
                "USD_INR": 83.0,
                "USD_JPY": 150.0
            }
            rate_key = f"{from_currency}_{to_currency}"
            rate = common_rates.get(rate_key, 1.0)
        
        converted = amount * rate
        return f"{amount} {from_currency} = {converted:.2f} {to_currency} (Rate: {rate})"

#### Multi-agent Workflow

In [11]:
class TravelAgentSystem:
    """Main travel agent system orchestrating all components"""
    
    def __init__(self):
        # Initialize LLM
        self.llm = ChatGroq(
            model_name="deepseek-r1-distill-llama-70b",
            temperature=0.3
        )
        
        # Collect all tools
        self.tools = [
            # Weather tools
            WeatherTools.get_current_weather,
            WeatherTools.get_weather_forecast,
            # Attraction tools
            AttractionTools.search_attractions,
            AttractionTools.search_restaurants,
            AttractionTools.search_activities,
            # Accommodation tools
            AccommodationTools.search_hotels,
            AccommodationTools.estimate_accommodation_cost,
            # Calculation tools
            CalculationTools.add_costs,
            CalculationTools.multiply_cost,
            CalculationTools.calculate_daily_budget,
            # Currency tools
            CurrencyTools.get_exchange_rate,
            CurrencyTools.convert_currency,
        ]
        
        # Bind tools to LLM
        self.llm_with_tools = self.llm.bind_tools(self.tools)
        
        # System prompt
        self.system_prompt = SystemMessage(
            content="""You are an expert AI Travel Agent and Expense Planner. Your goal is to create comprehensive travel plans with accurate cost estimates.

WORKFLOW:
1. Search for attractions, restaurants, and activities
2. Get weather information (current and forecast)
3. Find accommodation options and estimate costs
4. Calculate total expenses and daily budgets
5. Convert currencies as needed
6. Generate detailed itinerary
7. Create comprehensive trip summary

GUIDELINES:
- Always provide realistic cost estimates
- Consider the user's budget range (budget/mid-range/luxury)
- Include practical travel advice
- Generate day-by-day itineraries
- Calculate costs for the specified number of travelers
- Be thorough but concise in recommendations
- Always end with a complete summary

Be helpful, accurate, and thorough in your planning!"""
        )
        
        # Build the agent graph
        self._build_agent_graph()
    
    def _build_agent_graph(self):
        """Build the multi-agent workflow graph"""
        
        def travel_agent_node(state: MessagesState):
            """Main travel agent processing node"""
            user_messages = state["messages"]
            input_messages = [self.system_prompt] + user_messages
            response = self.llm_with_tools.invoke(input_messages)
            return {"messages": [response]}
        
        # Create the state graph
        workflow = StateGraph(MessagesState)
        workflow.add_node("travel_agent", travel_agent_node)
        workflow.add_node("tools", ToolNode(self.tools))
        
        # Define workflow edges
        workflow.add_edge(START, "travel_agent")
        workflow.add_conditional_edges(
            "travel_agent",
            tools_condition,
        )
        workflow.add_edge("tools", "travel_agent")
        
        # Compile the workflow
        self.agent_graph = workflow.compile()
    
    def plan_trip(self, travel_request: str) -> Dict:
        """
        Main method to plan a trip based on user request.
        
        Args:
            travel_request (str): User's travel planning request
            
        Returns:
            Dict: Complete travel plan response
        """
        message = [HumanMessage(content=travel_request)]
        response = self.agent_graph.invoke({"messages": message})
        return response
    
    

In [12]:
def main():
    """Example usage of the Travel Agent System"""
    
    # Initialize the travel agent system
    travel_system = TravelAgentSystem()
    
    # Input travel requests
    sample_requests = [
                
        """
        Plan a 7-day luxury trip to Tokyo, Japan for 1 person.
        I want to experience authentic culture, visit temples, try local cuisine.
        Budget is luxury level. Convert all costs to GBP.
        Include weather info and complete itinerary.
        """
    ]
    
    print("=== AI TRAVEL AGENT & EXPENSE PLANNER ===\n")
    
    # Process first sample request
    print("Processing travel request...")
    print("-" * 50)
    
    response = travel_system.plan_trip(sample_requests[0])
    
    # Display the conversation
    for message in response["messages"]:
        if hasattr(message, 'content'):
            print(f"Agent Response:")
            print(message.content)
            print("-" * 50)
    

In [14]:
if __name__ == "__main__":
    
    main()
    

=== AI TRAVEL AGENT & EXPENSE PLANNER ===

Processing travel request...
--------------------------------------------------
Agent Response:

        Plan a 7-day luxury trip to Tokyo, Japan for 1 person.
        I want to experience authentic culture, visit temples, try local cuisine.
        Budget is luxury level. Convert all costs to GBP.
        Include weather info and complete itinerary.
        
--------------------------------------------------
Agent Response:

--------------------------------------------------
Agent Response:
Current weather in Tokyo: Sprinkles. Fog. (Weather station: Tokyo Heliport, Japan). See more current weather Tokyo, Japan - Current temperature and weather conditions. Detailed hourly weather forecast for today - including wea
--------------------------------------------------
Agent Response:

--------------------------------------------------
Agent Response:
Weather forecast for Tokyo (7 days): Tokyo Extended Forecast with high and low temperatures. °F. L